In [12]:
import chess.pgn
from stockfish import Stockfish


In [13]:
file_path = '../../dados/experimentos/'
file_name = 'Morphy.pgn'

stockfish = Stockfish()
stockfish.set_depth(20)
stockfish.set_skill_level(20)


with open(f'{file_path}{file_name}') as pgn_file:
    game = chess.pgn.read_game(pgn_file)
    

